## word2vec으로 추천 시스템 구현하기 

In [1]:
from gensim.models import Word2Vec
import pandas as pd
from collections import defaultdict
import numpy as np
import mykmeans as kmeans

In [687]:
## string to int with ignore none disit
def toint(s,default=0):
    converted = default
    try :
        converted = int(s)
    except:
        #print('string convert exception "{0}" (default={1})"'.format(s , default))
        pass
    return converted

class m2v_recsys:
    model = None
    home_dir = '/Users/goodvc/Documents/data-analytics/movie-recommendation/'
    w2v_env = { 'min_count':5, 'size':100, 'window':5 }
    nn_func = None

    ## initialize
    def __init__(self, ds_type='ml-latest-small'):
        self.ds_type = ds_type
        self.movieLensDataLoad()

    ## dataset load
    def movieLensDataLoad(self, ds_type=None):
        if ds_type != None:§
    def training(self):
        if self.model == None :
            self.scentences = self.make_sentences()
            self.model = Word2Vec(self.scentences, 
                                  min_count = self.w2v_env['min_count'],
                                  size = self.w2v_env['size'], 
                                  window = self.w2v_env['window'])
            print('model trained')
            
    def nearest_neighbors_by_m2v(self, target, topn=10):
        if str == type(target):
            target=[target]
        result = []
        try:
            result = self.model.most_similar(positive=target, topn=topn)
        except:
            pass
        return result

    def user_interests(self, userId):
        return self.ratings_movie[self.ratings_movie.userId==userId][['title','rating']].values.tolist()
    
    def cal_score(self, title, sim, rating, pre_score):
        score = np.linalg.norm([sim, rating/5])
        score = score+pre_score
        return score
    
    def recommend_movies(self, userId, topn=10):
        movies = defaultdict(float)
        interests = self.user_interests(userId)
        seenMovies = set( m for m,s in interests )
        for title, rating in interests:
            neighbors = self.nearest_neighbors_by_m2v(title)
            for (title2, sim) in neighbors:
                if title2 in seenMovies:
                    continue
                pre = movies.get(title2, 0.0)
                movies[title2] = self.cal_score( title2, sim, rating, pre ) 
        return {'recommended' : sorted(movies.items(), key=lambda x: x[1], reverse=True)[:topn]
                ,'ratings':interests}

    ####################################
    ## clustering methods
    def clustering(self, k=100, delta=0.00000001, maxiter=100):
        self.movie_vec = self.model.syn0
        centres, index2cid, dist = kmeans.kmeanssample(self.movie_vec, k, 
                                                       metric = 'cosine', 
                                                       delta = delta, 
                                                       nsample = 0, maxiter = maxiter,)
        self.clustered_ds = pd.DataFrame( [ (a, b, c) for a, b, c in zip(self.model.index2word, index2cid, dist )],
                     columns=['title', 'cid', 'dist'] ).sort(['cid','dist'], ascending=True)
        
        self.movie2cid = { movie:cid for movie,cid in zip(self.model.index2word, index2cid) }
        
        return (centres, index2cid, dist)
    
    def clusterMembersByCId(self, cid):
        return self.clustered_ds[self.clustered_ds.cid==cid]
    
    def clusterMembersByTitle(self, title):
        cid = self.clustered_ds[self.clustered_ds.title==title]['cid'].values[0]
        return self.clusterMembersByCId(cid) 
    
    def clusterTags(self, cid):
        ds = pd.merge(self.tags
                 , self.clusterMembersByCId(cid)
                 , on='title')
        ds['dist'] = 1 - ds['dist']
        cid_tags = ds.groupby(['tag'])['dist'].sum().sort(inplace=False,ascending=False)[:5]
        return cid_tags
    ######################################
    
    def user_mcate_interests(self, uid, v=False):
        interests = self.user_interests(uid)
        cid_interests = defaultdict(int)
        total = 0
        for title, rating in interests:
            cid_interests[self.movie2cid.get(title, -1)] += rating/5
            total += (rating/5)
        interests_ds = pd.DataFrame. from_dict({'score':cid_interests},orient='columns')
        interests_ds['score'] = round(interests_ds['score'] / total,5)
        interests_ds.sort(['score'], ascending=False, inplace=True)
        
        if True==v:
            for cid, score in interests_ds.iterrows():
                print(cid,score.values[0],cid,self.clusterTags(cid).index.values[:3])
        return interests_ds

In [688]:
## 20m dataset
m2v_rs_ml20 = m2v_recsys('ml-20m')
m2v_rs_ml20.load_model('./resource/m2v_rs.ml20.model')

model loaded ./resource/m2v_rs.ml20.model


In [689]:
(centres, index2cid, dist ) = m2v_rs_ml20.clustering(k=500, maxiter=200, delta=1e-20)

kmeans: X (5000, 100)  centres (500, 100)  delta=1e-20  maxiter=200  metric=cosine
kmeans: 15 iterations  cluster sizes: [10  9  1  3  8  1  1 17 11  9  4  5 18  4  1  1  3  1  7 10 11 23 13 28 20
  2  6 18  3  7 23 14 17 12  8 33 15  7  4 12  4  1  3  2  2 16  9 15  7  1
 14 11  8  1  6  9 25  6  5  1  2  8 12  1 16  8 17 13 12 10  3 12  0  0 10
  8  6  0 13  1  0 12 25  1  1 18  4  1  6  4  6 26 54  7  5  4  5 27  7  9
  2 13  3  5 12  1  3  9  6  8  7 10  8  8 13 43  1  4  3  3 20 21  6 18 15
 10  1 14  5  5  2  1 12  2  3  1 15 19 12  9 16 28 15 23  7 12  0  8 20 30
  5  5  4  1 11  8 20 13  5  3  5  2 31  4 23 21 20  3  8  4 14 41  8  8  8
  2  9 24  6  7  0  9  1 26  6  4 10  1  7  2 19  3 10  0  1 15 16  3  6  1
  9  1  5 20  3  7 11 29  5 43  2  4 10  9 32  7 25 10 17  2 25 10 16 13 28
  5  5 23 18 19  8 14  3  1  4  9 11  9  1 16  3 11 17  0  7 16 15  2  5  3
  8  1  2  2 30 11 11 33 12  0  7  3  3 18  4  7  0  4  0 13 61 11 15  0 18
 12  7 10 22 35 14  9 23  6 14  8  2 19  0 

In [612]:
m2v_rs_ml20.clusterMembersByCId(23)

,title,cid,dist
5370,"Pursuit of D.B. Cooper, The (a.k.a. Pursuit)",23,0.038943
6233,War and Peace,23,0.045101
11859,"Monster Club, The",23,0.046061
4724,Martin,23,0.046308
12923,Cast a Giant Shadow,23,0.046832
1821,Ffolkes,23,0.049292
10218,"Ninth Configuration, The (a.k.a. Twinkle, Twin...",23,0.050250
8473,"Summer Place, A",23,0.053552
1214,"Black Scorpion, The",23,0.054639
12716,Mr. Hobbs Takes a Vacation,23,0.060728


In [603]:
m2v_rs_ml20.clusterMembersByTitle('Saw II')

,title,cid,dist
162,Saw IV,377,0.053128
11419,Saw V,377,0.062632
18,Saw III,377,0.087176
10089,Saw VI,377,0.089147
4585,Hostel: Part II,377,0.103229
8026,Saw II,377,0.134398
5553,Saw VII 3D - The Final Chapter,377,0.136295
7035,Scary Movie 4,377,0.253404


In [596]:
m2v_rs_ml20.clusterMembersByTitle('Nixon')

,title,cid,dist
7309,Leaving Las Vegas,103,0.103979
437,Dead Man Walking,103,0.107285
8919,Sense and Sensibility,103,0.130243
10656,Mr. Holland's Opus,103,0.147282
10023,Heat,103,0.176812
9431,Mighty Aphrodite,103,0.190162
1041,"Birdcage, The",103,0.190465
12173,Nixon,103,0.199732
13129,Primal Fear,103,0.257689
9578,Broken Arrow,103,0.319940


In [621]:
for cid in range(100,130):
    print('cid :',cid,m2v_rs_ml20.clusterTags(cid).index.values[:3])

cid : 100 ['Betamax' 'Bruce Lee' 'soccer']
cid : 101 ['sci-fi' 'action' 'Leonardo DiCaprio']
cid : 102 ['nudity (topless - notable)' 'Nudity (Full Frontal)'
 'not available from Netflix']
cid : 103 ['based on a book' 'atmospheric' 'Jane Austen']
cid : 104 ['sci-fi' 'classic' 'time travel']
cid : 105 ['Mama' 'Inception' 'Funny']
cid : 106 ['comedy' 'Will Ferrell' 'jesus']
cid : 107 ['less than 300 ratings'
 'Florida Film Festival Winner Special Jury Award for Narrative Filmmaking 1999'
 'Florida Film Festival Winner']
cid : 108 ['road trip' 'Nicole Kidman' 'Johnny Depp']
cid : 109 ['teacher student relationship' 'Pedro AlmodÃ³var' 'AlmodÃƒÂ³var!!!!']
cid : 110 ['Robert De Niro' 'Quentin Tarantino' 'mentor']
cid : 111 ['hallucinatory' 'David Lynch' 'surreal']
cid : 112 ['horror' 'Betamax' 'slasher']
cid : 113 ['revenge' 'psychology' 'predictable']
cid : 114 ['ninja']
cid : 115 []
cid : 116 ['time travel' 'Nudity (Topless)' 'supernatural']
cid : 117 ['noir thriller' 'CLV' 'neo-noir']
cid 

In [605]:
m2v_rs_ml20.clusterMembersByCId(90)

,title,cid,dist
11983,Pirates Who Don't Do Anything: A VeggieTales M...,90,0.023139
1873,Full of It,90,0.056454
11161,Billabong Odyssey,90,0.076041
13089,Baggage Claim,90,0.076166
1791,Hell Comes to Frogtown,90,0.076448
10915,MacGyver: Lost Treasure of Atlantis,90,0.087512
1031,"Pebble and the Penguin, The",90,0.090192
4074,Killer Bean 2: The Party,90,0.092154
3504,Super Sucker,90,0.092339
12435,"Blackout, The",90,0.094893


In [606]:
m2v_rs_ml20.clusterMembersByTitle('Matrix, The')

,title,cid,dist
3000,"Sixth Sense, The",342,0.145427
8826,"Matrix, The",342,0.146262
3664,Saving Private Ryan,342,0.166291
8681,"Princess Bride, The",342,0.209890
13095,L.A. Confidential,342,0.230251
11742,American Beauty,342,0.233694
12119,Good Will Hunting,342,0.247404
10507,Monty Python and the Holy Grail,342,0.250881
11952,Fight Club,342,0.265294
6461,Raiders of the Lost Ark (Indiana Jones and the...,342,0.280892


In [497]:
m2v_rs.clustering()

kmeans: X (1000, 100)  centres (100, 100)  delta=1e-08  maxiter=100  metric=cosine
kmeans: 11 iterations  cluster sizes: [ 4  8  4  9  5 10  5 24 11 86  3  5  4  3  6  4 10  2  5  9  5 51  3  9 25
  4 16  2 11  6  6  5 15  4  7 10  4  8  7 15  6  5  4  5  3  9  6  7 22 14
  8 10  6  5  7  9 64 12  3  9 12  3  4  3  8 11 32  4  4  7  2  9  6 10 11
  3  6 31  9  6 10  8 16  7  8  6 11  6 16  6 14  7  9  7  8  5  7  9  8  7]
kmeans: X (1099, 100)  centres (100, 100)  delta=1e-08  maxiter=100  metric=cosine
kmeans: 6 iterations  cluster sizes: [  5   8   5  10   6  12   5  23  12 101   3   8   4   5   7   5  10   5
   5  10   5  57   4  11  25   5  17   2  11   7   7   6  16   4   7  11
   5   8   7  16   8   6   6   5   3  10   6   7  23  15   8  10   6   5
  10   9  66  12   3  10  13   4   4   3   8  11  33   4   4   8   2  10
   6  14  11   5   6  34   9   6  13   9  18   8   9   6  12   6  17   6
  14   8  11   8   8   5   7  13   9   9]


(array([ 9,  9,  5, ..., 10, 56, 88]),
 array([[ -2.05094717e-03,   9.53356270e-04,   9.24346023e-05, ...,
          -1.03916787e-03,   1.68464263e-03,   1.30630215e-03],
        [  2.97280902e-04,  -1.09126291e-03,   1.41588598e-03, ...,
          -1.52803003e-03,  -1.86778745e-03,   8.85844813e-04],
        [  3.26073426e-03,  -1.84293871e-03,   1.99179118e-03, ...,
           8.54265410e-04,   2.14851781e-04,  -3.81570775e-04],
        ..., 
        [  1.34107075e-03,   1.44365162e-03,   4.71084175e-04, ...,
           2.47493805e-03,   9.32482071e-04,  -3.76198877e-04],
        [ -1.88021036e-03,  -2.52586673e-04,  -3.20180971e-03, ...,
          -2.15061867e-04,  -1.76366209e-03,   1.82774977e-03],
        [  2.44270195e-03,   1.25406054e-03,   2.93661375e-03, ...,
           2.71253451e-03,   2.64538336e-03,  -4.71767766e-04]], dtype=float32))

In [629]:
m2v_rs_ml20.clusterTags(81)

tag
stupid              8.137575
high school         7.826824
Robert Rodriguez    5.926682
Crappy Remake       4.857144
based on a book     4.464471
Name: dist, dtype: float64

In [666]:
interests = m2v_rs_ml20.user_interests(10)
cid_interests = defaultdict(int)
total = 0
for title, rating in interests:
    cid_interests[m2v_rs_ml20.movie2cid.get(title, -1)] += rating/5
    total += (rating/5)
interests_ds = pd.DataFrame. from_dict({'score':cid_interests},orient='columns')
interests_ds['score'] = (interests_ds['score'] / total)
interests_ds.sort(['score'], ascending=False, inplace=True)

#for cid in cid_interests.keys():
#    cid_interests[cid] = round(cid_interests[cid]/total,5)

,score
487,0.222973
480,0.189189
299,0.101351
125,0.087838
441,0.074324
322,0.054054
452,0.047297
105,0.027027
265,0.027027
277,0.027027


In [670]:
m2v_rs_ml20.user_mcate_interests(1)

,score
15,0.250000
363,0.162162
275,0.128378
226,0.094595
5,0.047297
142,0.047297
235,0.047297
143,0.027027
154,0.027027
212,0.027027


In [690]:
interests = m2v_rs_ml20.user_mcate_interests(100, v=True)

TypeError: type Series doesn't define __round__ method

In [691]:
for cid, score in interests.iterrows():
    print(cid,score.values[0],cid,m2v_rs_ml20.clusterTags(cid).index.values[:3])

280 0.128378378378 280 ['horror' 'Betamax' 'slasher']
128 0.121621621622 128 ['nostalgic' 'espionage' 'karate']
296 0.108108108108 296 ['Betamax' 'nudity (topless)' 'BD-R']
251 0.0743243243243 251 []
52 0.0608108108108 52 ['BD-R' 'less than 300 ratings' 'blaxploitation']
79 0.0608108108108 79 ['stranded' 'wartime' 'soldier']
5 0.0540540540541 5 ['less than 300 ratings' 'lawyer as protagonist?' 'Legal']
101 0.0540540540541 101 ['coming of age' 'quirky' 'stylized']
481 0.0472972972973 481 ['Hugh Dancy' 'Rose Byrne' 'asperger syndrome']
413 0.0405405405405 413 ['Hulk Hogan' 'franchise' 'Thanatos!']
126 0.027027027027 126 ['Buster Keaton' 'DVD-R' 'Eddie Izzard']
6 0.027027027027 6 ['Howard Hughes' 'perrot library' 'motorcycle']
110 0.027027027027 110 ['drugs' 'Johnny Depp' 'poker']
278 0.027027027027 278 ['funny' 'beautiful' 'cliche']
292 0.027027027027 292 ['treasure' 'family' 'supernatural']
376 0.027027027027 376 ['superhero' 'Johnny Depp' 'Will Smith']
437 0.027027027027 437 []
86 0.02

In [674]:
interests

,score
15,0.250000
363,0.162162
275,0.128378
226,0.094595
5,0.047297
142,0.047297
235,0.047297
143,0.027027
154,0.027027
212,0.027027
